In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import string 
import numpy as np
from string import punctuation
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
import torch

In [3]:
raw_articles = pd.read_csv("data/articles.csv")
raw_customers = pd.read_csv("data/customers.csv")
raw_transactions = pd.read_csv("data/transactions_train.csv")

In [4]:
articles = raw_articles.copy()
customers = raw_customers.copy()
transactions = raw_transactions.copy()

In [5]:
articles.head().T

,0,1,2,3,4
article_id,108775015,108775044,108775051,110065001,110065002
product_code,108775,108775,108775,110065,110065
prod_name,Strap top,Strap top,Strap top (1),OP T-shirt (Idro),OP T-shirt (Idro)
product_type_no,253,253,253,306,306
product_type_name,Vest top,Vest top,Vest top,Bra,Bra
product_group_name,Garment Upper body,Garment Upper body,Garment Upper body,Underwear,Underwear
graphical_appearance_no,1010016,1010016,1010017,1010016,1010016
graphical_appearance_name,Solid,Solid,Stripe,Solid,Solid
colour_group_code,9,10,11,9,10
colour_group_name,Black,White,Off White,Black,White


In [6]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [7]:
articles = articles.dropna()

In [8]:
articles["text"] = articles["prod_name"].map(str) + " " + articles["product_type_name"] +" "+ articles["product_group_name"]+ " "+ articles['graphical_appearance_name']+" "+ articles['colour_group_name'] +" "+ articles['perceived_colour_value_name']+ " " + articles["perceived_colour_master_name"] +" "+ articles["department_name"]+ " "+ articles['index_name']+" "+articles['index_group_name'] +" "+articles['section_name']+ " "+ articles['garment_group_name']+" "+articles['detail_desc']

In [9]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105126 entries, 0 to 105541
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105126 non-null  int64 
 1   product_code                  105126 non-null  int64 
 2   prod_name                     105126 non-null  object
 3   product_type_no               105126 non-null  int64 
 4   product_type_name             105126 non-null  object
 5   product_group_name            105126 non-null  object
 6   graphical_appearance_no       105126 non-null  int64 
 7   graphical_appearance_name     105126 non-null  object
 8   colour_group_code             105126 non-null  int64 
 9   colour_group_name             105126 non-null  object
 10  perceived_colour_value_id     105126 non-null  int64 
 11  perceived_colour_value_name   105126 non-null  object
 12  perceived_colour_master_id    105126 non-null  int64 
 13 

In [10]:
df_all = articles[['article_id', 'product_code', 'text']]

In [11]:
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [12]:
df_all['text'] = df_all['text'].apply(clean_txt)

In [13]:
df_all['text'].head()

0    strap top vest top garment upper body solid bl...
1    strap top vest top garment upper body solid wh...
2    strap top vest top garment upper body stripe w...
3    shirt idro bra underwear solid black dark blac...
4    shirt idro bra underwear solid white light whi...
Name: text, dtype: object

In [14]:
#!pip install sentence-transformers

In [15]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [16]:
text_list = df_all['text'].tolist()

sentence_embeddings  = sbert_model.encode(text_list[0])
sentence_embeddings.shape

(768,)

In [17]:
text_list[0:5]

['strap top vest top garment upper body solid black dark black jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body solid white light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body stripe white dusty light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'shirt idro bra underwear solid black dark black clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape bust provide good support narrow adjustable shoulder strap narrow hook eye fasten back without visible seam greater comfort',
 'shirt idro bra underwear solid white light white clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape

In [18]:
article_sentences = ["".join(article) for article in text_list]

In [19]:
article_sentences[:7]

['strap top vest top garment upper body solid black dark black jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body solid white light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'strap top vest top garment upper body stripe white dusty light white jersey basic ladieswear ladieswear womens everyday basics jersey basic jersey top narrow shoulder strap',
 'shirt idro bra underwear solid black dark black clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape bust provide good support narrow adjustable shoulder strap narrow hook eye fasten back without visible seam greater comfort',
 'shirt idro bra underwear solid white light white clean lingerie lingeries tights ladieswear womens lingerie nightwear microfibre shirt bra underwired mould lightly pad cup shape

In [20]:
#result = sbert_model.encode(article_sentences)

In [21]:
#result.shape

In [22]:
#np.save("embedding",result)

In [138]:
result = np.load("data/bert_embedding.npy")

In [139]:
result

array([[ 0.07695773, -0.7046743 , -1.3352145 , ..., -0.28840092,
        -0.18515919, -0.48897102],
       [ 0.11474682, -0.5391601 , -1.4026517 , ...,  0.32394218,
        -0.0243687 , -0.13027613],
       [ 0.16699375, -0.7935319 , -1.4112424 , ...,  0.33043057,
         0.03060157, -0.26536635],
       ...,
       [-0.27468535, -0.8611819 , -1.327831  , ..., -0.8209244 ,
        -0.11368735, -0.26305622],
       [ 0.09037471,  0.18992746, -0.8631339 , ..., -1.0456095 ,
         0.04710359,  0.16714016],
       [ 0.02211573, -0.8925    , -1.4527102 , ...,  0.15218817,
        -0.28289533, -0.27992767]], dtype=float32)

In [129]:
result.shape

(105126, 768)

In [26]:
transactions = transactions.dropna()

In [27]:
transactions =  transactions.sort_values(by='customer_id')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
14479579,2019-07-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,745232001,0.021169,1
23574622,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,890498002,0.031763,2
23574621,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,887593002,0.021593,2
23574620,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,841260003,0.011508,2
4212358,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,0.044051,1


In [28]:
merged_df = df_all.merge(transactions, how = 'inner', on = ['article_id'])

In [29]:
merged_df.head()

,article_id,product_code,text,t_dat,customer_id,price,sales_channel_id
0,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
1,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
2,108775015,108775,strap top vest top garment upper body solid bl...,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1
3,108775015,108775,strap top vest top garment upper body solid bl...,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,0.006763,1
4,108775015,108775,strap top vest top garment upper body solid bl...,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,0.006763,1


In [30]:
merged_df2 = merged_df.groupby('customer_id', sort=False)['text'].apply(' '.join).reset_index()
merged_df2.head(5)

,customer_id,text
0,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,strap top vest top garment upper body solid bl...
1,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,strap top vest top garment upper body solid bl...
2,001ba9e81e13ce12a2585d9ebde923fe74429e9e12ea59...,strap top vest top garment upper body solid bl...
3,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,strap top vest top garment upper body solid bl...
4,002323971cbd38fad4512d5114676e5e17eb262db02320...,strap top vest top garment upper body solid bl...


In [31]:
u = "000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318" #customer_id
index = np.where(merged_df2['customer_id'] == u)[0][0]
cust_q = merged_df2.iloc[[index]]
cust_q

,customer_id,text
317613,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,lazer razer brief swimwear bottom swimwear sol...


In [32]:
transactions.loc[transactions['customer_id'] == u]

,t_dat,customer_id,article_id,price,sales_channel_id
24773518,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,858883002,0.030492,2
24773520,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,0.042356,2
24773521,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,750424014,0.042356,2
24773522,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,870304002,0.033881,2
24773523,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,870304002,0.033881,2
24773524,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,852643001,0.025407,2
24773525,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,852643003,0.025407,2
21953952,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,351484002,0.022017,2
21953951,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,351484002,0.022017,2
24773519,2020-04-18,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,851400006,0.059305,2


In [33]:
def recommendation_product(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['customer_id', 'article_id',  'product_code', 'detail_desc', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'customer_id'] = u
        recommendation.at[count, 'article_id'] = df_all['article_id'][i]
        recommendation.at[count, 'product_code'] = df_all['product_code'][i]
        recommendation.at[count, 'detail_desc'] = articles['detail_desc'][i]   
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    #print("recommendations generated")        
    return recommendation

In [178]:
cust_list = cust_q['text'].tolist()
user_embeddings  = sbert_model.encode(cust_list)
user_embeddings

array([[-1.27318844e-01, -7.91413903e-01, -1.35676587e+00,
         4.66501534e-01,  8.91606450e-01, -3.00245494e-01,
         3.74543369e-02, -5.74706554e-01,  3.51013839e-01,
         3.74542087e-01,  5.77386737e-01,  8.73587132e-01,
        -3.08815181e-01,  7.75455683e-02, -1.09578192e-01,
         2.50823438e-01,  5.90778768e-01,  1.66480662e-03,
         2.81401098e-01,  1.94867164e-01, -6.25387371e-01,
        -1.88409358e-01,  5.58481663e-02, -4.82794076e-01,
         1.02236915e+00,  8.58962893e-01,  1.48215204e-01,
         8.61553609e-01,  1.50197148e-01,  5.29901087e-01,
         6.07713163e-01,  5.95571995e-01, -4.13479775e-01,
         5.05680144e-01,  1.15274382e+00,  8.59117746e-01,
        -2.11196095e-02, -2.66492367e-01,  1.91199124e-01,
         2.77442455e-01,  2.03172088e-01, -7.97289193e-01,
         9.26105157e-02, -3.83972734e-01, -1.17017472e+00,
        -5.89617312e-01, -8.21041822e-01,  5.11348188e-01,
        -2.75748670e-02, -3.13990295e-01, -1.12762619e-0

In [179]:
user_embeddings.shape

(1, 768)

In [182]:
user_embeddings = user_embeddings.reshape(-1,1)
user_embeddings

array([[-1.27318844e-01],
       [-7.91413903e-01],
       [-1.35676587e+00],
       [ 4.66501534e-01],
       [ 8.91606450e-01],
       [-3.00245494e-01],
       [ 3.74543369e-02],
       [-5.74706554e-01],
       [ 3.51013839e-01],
       [ 3.74542087e-01],
       [ 5.77386737e-01],
       [ 8.73587132e-01],
       [-3.08815181e-01],
       [ 7.75455683e-02],
       [-1.09578192e-01],
       [ 2.50823438e-01],
       [ 5.90778768e-01],
       [ 1.66480662e-03],
       [ 2.81401098e-01],
       [ 1.94867164e-01],
       [-6.25387371e-01],
       [-1.88409358e-01],
       [ 5.58481663e-02],
       [-4.82794076e-01],
       [ 1.02236915e+00],
       [ 8.58962893e-01],
       [ 1.48215204e-01],
       [ 8.61553609e-01],
       [ 1.50197148e-01],
       [ 5.29901087e-01],
       [ 6.07713163e-01],
       [ 5.95571995e-01],
       [-4.13479775e-01],
       [ 5.05680144e-01],
       [ 1.15274382e+00],
       [ 8.59117746e-01],
       [-2.11196095e-02],
       [-2.66492367e-01],
       [ 1.9

In [183]:
user_embeddings.shape

(768, 1)

In [38]:
# mask = np.zeros(result.shape,dtype='uint8')
# mask.shape

(105126, 768)

In [39]:
#similarities = cosine_similarity(result)

MemoryError: Unable to allocate 41.2 GiB for an array with shape (105126, 105126) and data type float32

In [ ]:
#print('pairwise dense output:\n {}\n'.format(similarities))

In [130]:
result.shape[0]

105126

In [131]:
result

array([[ 0.07695773, -0.7046743 , -1.3352145 , ..., -0.28840092,
        -0.18515919, -0.48897102],
       [ 0.11474682, -0.5391601 , -1.4026517 , ...,  0.32394218,
        -0.0243687 , -0.13027613],
       [ 0.16699375, -0.7935319 , -1.4112424 , ...,  0.33043057,
         0.03060157, -0.26536635],
       ...,
       [-0.27468535, -0.8611819 , -1.327831  , ..., -0.8209244 ,
        -0.11368735, -0.26305622],
       [ 0.09037471,  0.18992746, -0.8631339 , ..., -1.0456095 ,
         0.04710359,  0.16714016],
       [ 0.02211573, -0.8925    , -1.4527102 , ...,  0.15218817,
        -0.28289533, -0.27992767]], dtype=float32)

In [72]:
result1 = result.astype(int)
cut_size = result1[0]/size
cut_size.dtype

500

In [87]:
# size = 500
# cut_size = 2503
# for i in range(105126):
#     if i % 20 == 0:
#         print('i={}'.format(i))
#     if i==0:
#         inputpiece = result[i* cut_size:(i+1)*cut_size,:]
#         #print("------------",inputpiece)
#         res = cosine_similarity(inputpiece)
#         index2=np.argpartition(res, kth=-1, axis=0)[-1:,:]
#     elif i==size-1:
#         inputpiece = result[i* cut_size:,:]
#         res = cosine_similarity(inputpiece)
#         index2=np.concate((index2, np.argpartition(res, kth=-1, axis=0)[-1:,:]), axis=1)
#     else:
#         inputpiece = result[i* cut_size:(i+1)*cut_size,:]
#         res = cosine_similarity(inputpiece)
#         index2=np.concatenate((index2, np.argpartition(res, kth=-1, axis=0)[-1:,:]), axis=1)

i=0
i=20
i=40


ValueError: Found array with 0 sample(s) (shape=(0, 768)) while a minimum of 1 is required by check_pairwise_arrays.

In [75]:
# size = 500
# cut_size = 2503
# i=0
# while(i<=105126):
#     if i==0:
#         inputpiece = result[i* cut_size:(i+1)*cut_size,:]
#         #print("------------",inputpiece)
#         res = cosine_similarity(inputpiece)
#         index2=np.argpartition(res, kth=-1, axis=0)[-1:,:]
#         i=2503
#     else:
#         i=250
#         inputpiece = result[i* cut_size:,:]
#         res = cosine_similarity(inputpiece)
#         index2=np.concate((index2, np.argpartition(res, kth=-1, axis=0)[-1:,:]), axis=1)
#     else:
#         inputpiece = result[i* cut_size:(i+1)*cut_size,:]
#         res = cosine_similarity(inputpiece)
#         index2=np.concatenate((index2, np.argpartition(res, kth=-1, axis=0)[-1:,:]), axis=1)

array([ 7.69577324e-02, -7.04674304e-01, -1.33521450e+00,  2.99998105e-01,
        1.07769346e+00,  1.65203527e-01,  5.64638257e-01, -2.27492362e-01,
        1.32548243e-01,  1.97122265e-02,  5.75560570e-01,  1.06553423e+00,
       -7.74935544e-01, -5.13326610e-04,  8.98681805e-02,  8.80451441e-01,
        2.55123794e-01, -1.03246443e-01, -5.46026409e-01,  4.11850840e-01,
       -1.35624567e-02, -1.62795991e-01,  1.12183702e+00, -6.89392865e-01,
        6.63944900e-01,  6.21687591e-01,  2.75615513e-01,  9.13625836e-01,
       -5.82062125e-01,  1.13605702e+00,  5.65348566e-01, -4.13133353e-01,
        3.05927433e-02, -1.58370465e-01,  1.22476768e+00,  2.32065961e-01,
       -1.03282459e-01,  5.42716265e-01,  6.27560616e-01, -1.14507997e+00,
        1.37995315e+00, -1.42193794e-01,  1.83060735e-01,  3.27181637e-01,
       -1.30528998e+00, -4.65152621e-01, -8.73382747e-01,  5.38944840e-01,
       -1.22879282e-01, -4.46092963e-01, -1.54284239e-01,  3.87162939e-02,
        1.53368509e+00,  

In [140]:
arr=[]
for i in range(0,105126):
    if i%2503==0:
        arr.append(i)

In [186]:
for i in range(0,42):
    if i==0:
        inputpiece = result[arr[i]:arr[i+1],:]
        res = cosine_similarity(inputpiece)
        index2=res
        #print(res.shape)
    else:
        inputpiece = result[arr[i+1]:arr[i+2],:]
        res = cosine_similarity(inputpiece)
        #print(res.shape)
        index2=np.concatenate((index2,res), axis=1)

IndexError: list index out of range

In [169]:
# unique, counts = np.unique(index2, return_counts=True)
# result = np.column_stack((unique, counts)) 
# print (result)

In [174]:
print(index2)

[[0.99999964 0.76325554 0.8152565  ... 0.5278139  0.7271378  0.49461615]
 [0.76325554 1.0000001  0.97251934 ... 0.4322164  0.6216578  0.54895085]
 [0.8152565  0.97251934 1.0000002  ... 0.5072907  0.72890365 0.5314543 ]
 ...
 [0.67612106 0.5036862  0.5712719  ... 0.99999964 0.73236454 0.76123965]
 [0.72522414 0.41267318 0.4922266  ... 0.73236454 1.0000001  0.7004309 ]
 [0.65485084 0.6233079  0.67182803 ... 0.76123965 0.7004309  0.99999994]]


In [ ]:
len(index2)

In [153]:
top = sorted(range(len(index2)), key=lambda i: index2[i], reverse=True)[:13]
tf_list_scores = [index2[i][0][0] for i in top]
recommendation_product(top, df_all, tf_list_scores)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [154]:
index2[0][0]

1.0000002

In [188]:
cos_similarity_bert = map(lambda x: cosine_similarity(user_embeddings, x),index2)
cos_similarity_bert

In [189]:
list(cos_similarity_bert)

ValueError: Expected 2D array, got 1D array instead:
array=[0.99999964 0.76325554 0.8152565  ... 0.5278139  0.7271378  0.49461615].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

NameError: name 'op' is not defined